# Tạo Dữ Liệu Giảng Viên cho Hệ Thống eUIT

Notebook này sẽ tạo dữ liệu mẫu cho bảng `giang_vien` trong cơ sở dữ liệu eUIT, bao gồm:

## Yêu cầu chính:
- **Mã giảng viên**: Format 80xxx (5 chữ số bắt đầu từ 80001)
- **Khoa/Bộ môn**: Dựa trên dữ liệu từ bảng môn học
- **CCCD**: 12 số theo quy tắc: mã tỉnh (3 số) + mã thế kỷ/giới tính (1 số) + năm sinh (2 số) + số ngẫu nhiên (6 số)
- **Năm sinh**: 1970-1990 (độ tuổi hợp lý cho giảng viên)
- **Địa chỉ**: Dựa trên danh mục xã phường sau sáp nhập
- **Phân bố theo khoa**: Dựa trên số lượng môn học của từng khoa

In [1]:
# Import Required Libraries and Setup
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, date, timedelta
import csv
import os
from typing import List, Dict, Tuple
from collections import Counter

# Set random seed for reproducible results
random.seed(42)
np.random.seed(42)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
# Load and analyze existing data
# Read mon_hoc data to get khoa_bo_mon information
mon_hoc_df = pd.read_csv('../main_data/danh_muc_mon_hoc.csv', sep=';', encoding='utf-8')
print("Danh sách các khoa/bộ môn từ dữ liệu môn học:")
khoa_bo_mon_counts = mon_hoc_df['Đơn vị quản lý chuyên môn'].value_counts()
print(khoa_bo_mon_counts)
print(f"\nTổng số khoa/bộ môn: {len(khoa_bo_mon_counts)}")

Danh sách các khoa/bộ môn từ dữ liệu môn học:
Đơn vị quản lý chuyên môn
HTTT     210
KHMT     171
KTMT     130
MMTTT    121
KTTT     106
CNPM      93
PĐTĐH     56
BMTL      28
TTNN      26
BMAV       9
MMT        7
GDQP       1
GDTC       1
Name: count, dtype: int64

Tổng số khoa/bộ môn: 13


In [4]:
# Load address data
try:
    dia_chi_df = pd.read_csv('../other_data/danh_muc_xa_phuong_sau_sap_nhap.csv', sep=';', encoding='utf-8')
    print(f"Loaded {len(dia_chi_df)} địa chỉ")
    print("Columns:", dia_chi_df.columns.tolist())
    print(dia_chi_df.head())
    
    # Rename columns for easier access
    dia_chi_df.columns = ['ma_phuong_xa', 'xa_phuong', 'tinh_thanh_pho', 'ma_tinh']
    print("\nAfter renaming columns:")
    print(dia_chi_df.head())
    
except Exception as e:
    print(f"Error loading address data: {e}")
    # Fallback to basic addresses
    basic_addresses = [
        "Quận 1, TP. Hồ Chí Minh",
        "Quận 3, TP. Hồ Chí Minh", 
        "Quận 5, TP. Hồ Chí Minh",
        "Quận 7, TP. Hồ Chí Minh",
        "Quận Thủ Đức, TP. Hồ Chí Minh"
    ]
    dia_chi_df = pd.DataFrame({
        'xa_phuong': basic_addresses,
        'quan_huyen': basic_addresses,
        'tinh_thanh_pho': ['TP. Hồ Chí Minh'] * len(basic_addresses)
    })

Loaded 3321 địa chỉ
Columns: ['Mã phường/xã mới ', 'Tên Phường/Xã mới', 'Tên tỉnh/TP mới', 'Mã tỉnh (TMS)']
   Mã phường/xã mới  Tên Phường/Xã mới   Tên tỉnh/TP mới  Mã tỉnh (TMS)
0           10105001  Phường Hoàn Kiếm  Thành phố Hà Nội            101
1           10105002    Phường Cửa Nam  Thành phố Hà Nội            101
2           10101003    Phường Ba Đình  Thành phố Hà Nội            101
3           10101004    Phường Ngọc Hà  Thành phố Hà Nội            101
4           10101005   Phường Giảng Võ  Thành phố Hà Nội            101

After renaming columns:
   ma_phuong_xa         xa_phuong    tinh_thanh_pho  ma_tinh
0      10105001  Phường Hoàn Kiếm  Thành phố Hà Nội      101
1      10105002    Phường Cửa Nam  Thành phố Hà Nội      101
2      10101003    Phường Ba Đình  Thành phố Hà Nội      101
3      10101004    Phường Ngọc Hà  Thành phố Hà Nội      101
4      10101005   Phường Giảng Võ  Thành phố Hà Nội      101


In [ ]:
# Configuration for teacher data generation
TOTAL_TEACHERS = 100
TEACHER_ID_START = 80001

# Age range for teachers (born between 1970-1990)
BIRTH_YEAR_RANGE = (1970, 1990)

# Vietnam provinces for CCCD (first 3 digits)
PROVINCE_CODES = {
    '001': 'Hà Nội',
    '079': 'TP. Hồ Chí Minh', 
    '048': 'Đà Nẵng',
    '092': 'Cần Thơ',
    '024': 'Bắc Giang',
    '027': 'Bắc Ninh',
    '031': 'Hải Dương',
    '033': 'Hải Phòng',
    '040': 'Hưng Yên',
    '058': 'Thái Nguyên'
}

# Religion and ethnicity options (Vietnamese context)
RELIGIONS = ['Không', 'Phật giáo', 'Công giáo', 'Cao Đài', 'Hòa Hảo', 'Tin Lành']
ETHNICITIES = ['Kinh', 'Hoa', 'Khmer', 'Tày', 'Nùng', 'Mường']

# CCCD issuing places
CCCD_PLACES = [
    'Cục CS ĐKQL cư trú và DLQG về dân cư',
    'Công an TP. Hồ Chí Minh',
    'Công an tỉnh Đồng Nai',
    'Công an tỉnh Bình Dương',
    'Công an tỉnh Long An'
]

print("Configuration loaded successfully!")

In [ ]:
# Calculate teacher distribution based on department workload
# More teachers for departments with more subjects
def calculate_teacher_distribution(khoa_counts: pd.Series, total_teachers: int) -> Dict[str, int]:
    """
    Calculate teacher distribution based on number of subjects per department
    """
    # Calculate proportional distribution
    total_subjects = khoa_counts.sum()
    distribution = {}
    
    for khoa, count in khoa_counts.items():
        # Calculate proportion but ensure minimum 1 teacher per department
        proportion = count / total_subjects
        teachers_needed = max(1, int(proportion * total_teachers))
        distribution[khoa] = teachers_needed
    
    # Adjust to exactly match total_teachers
    current_total = sum(distribution.values())
    
    if current_total < total_teachers:
        # Add teachers to largest departments
        sorted_depts = sorted(distribution.items(), key=lambda x: x[1], reverse=True)
        for i in range(total_teachers - current_total):
            dept = sorted_depts[i % len(sorted_depts)][0]
            distribution[dept] += 1
    elif current_total > total_teachers:
        # Remove teachers from largest departments (but keep minimum 1)
        sorted_depts = sorted(distribution.items(), key=lambda x: x[1], reverse=True)
        for i in range(current_total - total_teachers):
            dept = sorted_depts[i % len(sorted_depts)][0]
            if distribution[dept] > 1:
                distribution[dept] -= 1
    
    return distribution

teacher_distribution = calculate_teacher_distribution(khoa_bo_mon_counts, TOTAL_TEACHERS)
print("Phân bố giảng viên theo khoa/bộ môn:")
for dept, count in sorted(teacher_distribution.items(), key=lambda x: x[1], reverse=True):
    print(f"{dept}: {count} giảng viên")

print(f"\nTổng: {sum(teacher_distribution.values())} giảng viên")

In [ ]:
# Vietnamese name lists for teachers
TEACHER_FIRST_NAMES_MALE = [
    'Nguyễn Văn', 'Trần Văn', 'Lê Văn', 'Phạm Văn', 'Hoàng Văn', 'Huỳnh Văn', 'Phan Văn',
    'Vũ Văn', 'Võ Văn', 'Đặng Văn', 'Bùi Văn', 'Đỗ Văn', 'Hồ Văn', 'Ngô Văn', 'Dương Văn',
    'Nguyễn Minh', 'Trần Minh', 'Lê Minh', 'Phạm Minh', 'Hoàng Minh', 'Huỳnh Minh',
    'Nguyễn Quang', 'Trần Quang', 'Lê Quang', 'Phạm Quang', 'Hoàng Quang',
    'Nguyễn Đức', 'Trần Đức', 'Lê Đức', 'Phạm Đức', 'Hoàng Đức',
    'Nguyễn Hữu', 'Trần Hữu', 'Lê Hữu', 'Phạm Hữu', 'Hoàng Hữu'
]

TEACHER_FIRST_NAMES_FEMALE = [
    'Nguyễn Thị', 'Trần Thị', 'Lê Thị', 'Phạm Thị', 'Hoàng Thị', 'Huỳnh Thị', 'Phan Thị',
    'Vũ Thị', 'Võ Thị', 'Đặng Thị', 'Bùi Thị', 'Đỗ Thị', 'Hồ Thị', 'Ngô Thị', 'Dương Thị',
    'Nguyễn Thùy', 'Trần Thùy', 'Lê Thùy', 'Phạm Thùy', 'Hoàng Thùy',
    'Nguyễn Thu', 'Trần Thu', 'Lê Thu', 'Phạm Thu', 'Hoàng Thu',
    'Nguyễn Minh', 'Trần Minh', 'Lê Minh', 'Phạm Minh', 'Hoàng Minh'
]

TEACHER_LAST_NAMES_MALE = [
    'An', 'Bình', 'Cường', 'Dũng', 'Đạt', 'Đức', 'Giang', 'Hải', 'Hiếu', 'Hùng',
    'Khang', 'Kiên', 'Long', 'Minh', 'Nam', 'Phong', 'Quang', 'Sơn', 'Thành', 'Tuấn',
    'Việt', 'Vinh', 'Vương', 'Xuân', 'Yên', 'Trung', 'Tùng', 'Thắng', 'Thịnh', 'Toàn',
    'Hưng', 'Hạnh', 'Hoàn', 'Huy', 'Lâm', 'Linh', 'Lộc', 'Phúc', 'Tài', 'Thọ'
]

TEACHER_LAST_NAMES_FEMALE = [
    'Anh', 'Bích', 'Chi', 'Diệu', 'Giang', 'Hà', 'Hạnh', 'Hiền', 'Hoa', 'Hương',
    'Lan', 'Linh', 'Mai', 'Nga', 'Nhung', 'Oanh', 'Phương', 'Quyên', 'Thảo', 'Thu',
    'Thúy', 'Trang', 'Tuyết', 'Uyên', 'Vân', 'Yến', 'Xuân', 'Hằng', 'Hồng', 'Huệ',
    'Kim', 'Lệ', 'Ly', 'My', 'Ngọc', 'Như', 'Thanh', 'Thùy', 'Tú', 'Vũ'
]

print(f"Loaded {len(TEACHER_FIRST_NAMES_MALE)} male first names")
print(f"Loaded {len(TEACHER_FIRST_NAMES_FEMALE)} female first names")
print(f"Loaded {len(TEACHER_LAST_NAMES_MALE)} male last names")
print(f"Loaded {len(TEACHER_LAST_NAMES_FEMALE)} female last names")

In [ ]:
# Helper functions for data generation
def generate_teacher_name(gender: str) -> str:
    """
    Generate a Vietnamese teacher name based on gender
    """
    if gender.lower() == 'nam':
        first_name = random.choice(TEACHER_FIRST_NAMES_MALE)
        last_name = random.choice(TEACHER_LAST_NAMES_MALE)
    else:
        first_name = random.choice(TEACHER_FIRST_NAMES_FEMALE)
        last_name = random.choice(TEACHER_LAST_NAMES_FEMALE)
    
    return f"{first_name} {last_name}"

def generate_cccd(birth_year: int, gender: str) -> str:
    """
    Generate a valid CCCD number
    Format: XXX Y ZZ NNNNNN
    XXX: Province code (3 digits)
    Y: Century and gender (1 digit) - 0/1 for 1900s, 2/3 for 2000s, odd for male, even for female
    ZZ: Last 2 digits of birth year
    NNNNNN: Random 6 digits
    """
    province_code = random.choice(list(PROVINCE_CODES.keys()))
    
    # Determine century and gender code
    if birth_year >= 2000:
        if gender.lower() == 'nam':
            century_gender = '3'
        else:
            century_gender = '2'
    else:
        if gender.lower() == 'nam':
            century_gender = '1'
        else:
            century_gender = '0'
    
    year_digits = str(birth_year)[-2:]
    random_digits = ''.join([str(random.randint(0, 9)) for _ in range(6)])
    
    return province_code + century_gender + year_digits + random_digits

def generate_phone_number() -> str:
    """
    Generate a Vietnamese mobile phone number
    """
    prefixes = ['090', '091', '094', '083', '084', '085', '081', '082', '088', '032', '033', '034', '035', '036', '037', '038', '039']
    prefix = random.choice(prefixes)
    suffix = ''.join([str(random.randint(0, 9)) for _ in range(7)])
    return prefix + suffix

def generate_email(full_name: str, teacher_id: str) -> str:
    """
    Generate email address for teacher
    """
    # Convert Vietnamese name to ASCII for email
    name_parts = full_name.lower().split()
    
    # Simple conversion (remove diacritics)
    conversions = {
        'à': 'a', 'á': 'a', 'ạ': 'a', 'ả': 'a', 'ã': 'a', 'â': 'a', 'ầ': 'a', 'ấ': 'a', 'ậ': 'a', 'ẩ': 'a', 'ẫ': 'a',
        'ă': 'a', 'ằ': 'a', 'ắ': 'a', 'ặ': 'a', 'ẳ': 'a', 'ẵ': 'a',
        'è': 'e', 'é': 'e', 'ẹ': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ê': 'e', 'ề': 'e', 'ế': 'e', 'ệ': 'e', 'ể': 'e', 'ễ': 'e',
        'ì': 'i', 'í': 'i', 'ị': 'i', 'ỉ': 'i', 'ĩ': 'i',
        'ò': 'o', 'ó': 'o', 'ọ': 'o', 'ỏ': 'o', 'õ': 'o', 'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ộ': 'o', 'ổ': 'o', 'ỗ': 'o',
        'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ợ': 'o', 'ở': 'o', 'ỡ': 'o',
        'ù': 'u', 'ú': 'u', 'ụ': 'u', 'ủ': 'u', 'ũ': 'u', 'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ự': 'u', 'ử': 'u', 'ữ': 'u',
        'ỳ': 'y', 'ý': 'y', 'ỵ': 'y', 'ỷ': 'y', 'ỹ': 'y',
        'đ': 'd'
    }
    
    email_name = ''
    for char in ' '.join(name_parts):
        email_name += conversions.get(char, char)
    
    email_name = email_name.replace(' ', '.')
    return f"{email_name}.{teacher_id[-3:]}@uit.edu.vn"

print("Helper functions defined successfully!")

In [ ]:
# Generate teacher data
def generate_teachers_data(distribution: Dict[str, int]) -> List[Dict]:
    """
    Generate teacher data based on department distribution
    """
    teachers_data = []
    current_id = TEACHER_ID_START
    
    for khoa_bo_mon, num_teachers in distribution.items():
        print(f"Generating {num_teachers} teachers for {khoa_bo_mon}...")
        
        for i in range(num_teachers):
            # Generate basic info
            gender = random.choice(['Nam', 'Nữ'])
            birth_year = random.randint(BIRTH_YEAR_RANGE[0], BIRTH_YEAR_RANGE[1])
            birth_month = random.randint(1, 12)
            birth_day = random.randint(1, 28)  # Safe day for all months
            birth_date = date(birth_year, birth_month, birth_day)
            
            # Generate name and IDs
            teacher_id = f"{current_id:05d}"
            full_name = generate_teacher_name(gender)
            cccd = generate_cccd(birth_year, gender)
            phone = generate_phone_number()
            
            # Generate CCCD issue date (after 18th birthday)
            cccd_issue_date = birth_date + timedelta(days=random.randint(18*365, 25*365))
            
            # Generate address
            address_row = dia_chi_df.sample(1).iloc[0]
            if 'xa_phuong' in address_row:
                phuong_xa = address_row['xa_phuong']
                tinh_thanh_pho = address_row.get('tinh_thanh_pho', 'TP. Hồ Chí Minh')
            else:
                phuong_xa = address_row.iloc[0]
                tinh_thanh_pho = 'TP. Hồ Chí Minh'
            
            # Generate detailed address
            house_number = random.randint(1, 999)
            street_names = ['Nguyễn Trãi', 'Lê Lợi', 'Hai Bà Trưng', 'Điện Biên Phủ', 'Cách Mạng Tháng 8']
            street = random.choice(street_names)
            detailed_address = f"{house_number} đường {street}, {phuong_xa}"
            
            # Generate other personal info
            ethnicity = random.choice(ETHNICITIES)
            religion = random.choice(RELIGIONS)
            cccd_place = random.choice(CCCD_PLACES)
            
            # Create teacher record
            teacher = {
                'ma_giang_vien': teacher_id,
                'ho_ten': full_name,
                'khoa_bo_mon': khoa_bo_mon,
                'ngay_sinh': birth_date.strftime('%Y-%m-%d'),
                'noi_sinh': f"{tinh_thanh_pho}",
                'cccd': cccd,
                'ngay_cap_cccd': cccd_issue_date.strftime('%Y-%m-%d'),
                'noi_cap_cccd': cccd_place,
                'dan_toc': ethnicity,
                'ton_giao': religion,
                'so_dien_thoai': phone,
                'dia_chi_thuong_tru': detailed_address,
                'tinh_thanh_pho': tinh_thanh_pho,
                'phuong_xa': phuong_xa
            }
            
            teachers_data.append(teacher)
            current_id += 1
    
    return teachers_data

# Generate the data
print("Starting teacher data generation...")
teachers = generate_teachers_data(teacher_distribution)
print(f"Generated {len(teachers)} teacher records")

In [ ]:
# Convert to DataFrame and display sample
teachers_df = pd.DataFrame(teachers)

print("Sample teacher data:")
print(teachers_df.head(10))

print(f"\nTotal teachers generated: {len(teachers_df)}")
print(f"Teachers by department:")
print(teachers_df['khoa_bo_mon'].value_counts())

In [ ]:
# Data validation and quality checks
print("=== DATA VALIDATION ===")

# Check for duplicates
print(f"Duplicate teacher IDs: {teachers_df['ma_giang_vien'].duplicated().sum()}")
print(f"Duplicate CCCDs: {teachers_df['cccd'].duplicated().sum()}")
print(f"Duplicate phone numbers: {teachers_df['so_dien_thoai'].duplicated().sum()}")

# Check ID format
invalid_ids = teachers_df[~teachers_df['ma_giang_vien'].str.match(r'^80\d{3}$')]
print(f"Invalid teacher ID format: {len(invalid_ids)}")

# Check CCCD format
invalid_cccd = teachers_df[teachers_df['cccd'].str.len() != 12]
print(f"Invalid CCCD length: {len(invalid_cccd)}")

# Check phone format
invalid_phone = teachers_df[teachers_df['so_dien_thoai'].str.len() != 10]
print(f"Invalid phone length: {len(invalid_phone)}")

# Check date validity
try:
    pd.to_datetime(teachers_df['ngay_sinh'])
    pd.to_datetime(teachers_df['ngay_cap_cccd'])
    print("All dates are valid")
except:
    print("Some dates are invalid")

print("\n=== SUMMARY STATISTICS ===")
birth_years = pd.to_datetime(teachers_df['ngay_sinh']).dt.year
print(f"Birth year range: {birth_years.min()} - {birth_years.max()}")
print(f"Average birth year: {birth_years.mean():.1f}")

print(f"\nGender distribution (estimated from names):")
male_names = teachers_df[teachers_df['ho_ten'].str.contains('Văn|Minh|Quang|Đức|Hữu')]['ho_ten'].count()
female_names = teachers_df[teachers_df['ho_ten'].str.contains('Thị|Thùy|Thu ')]['ho_ten'].count()
print(f"Male (estimated): {male_names}")
print(f"Female (estimated): {female_names}")
print(f"Other/Ambiguous: {len(teachers_df) - male_names - female_names}")

In [ ]:
# Export to CSV
output_file = '../main_data/giang_vien.csv'

# Ensure directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

# Export with proper encoding
teachers_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"Teacher data exported to: {output_file}")
print(f"File size: {os.path.getsize(output_file) / 1024:.1f} KB")

# Display final summary
print(f"\n=== EXPORT SUMMARY ===")
print(f"Total teachers: {len(teachers_df)}")
print(f"Departments covered: {teachers_df['khoa_bo_mon'].nunique()}")
print(f"Teacher ID range: {teachers_df['ma_giang_vien'].min()} - {teachers_df['ma_giang_vien'].max()}")
print(f"")
print("Teachers by department:")
for dept, count in teachers_df['khoa_bo_mon'].value_counts().items():
    print(f"  {dept}: {count}")

In [ ]:
# Generate SQL INSERT statements
def generate_sql_inserts(df: pd.DataFrame, table_name: str = 'giang_vien') -> str:
    """
    Generate SQL INSERT statements for the teacher data
    """
    sql_statements = []
    sql_statements.append(f"-- INSERT statements for {table_name} table")
    sql_statements.append(f"-- Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    sql_statements.append(f"-- Total records: {len(df)}")
    sql_statements.append("")
    
    # Add batch insert for better performance
    batch_size = 50
    for i in range(0, len(df), batch_size):
        batch_df = df.iloc[i:i+batch_size]
        
        sql_statements.append(f"-- Batch {i//batch_size + 1}")
        sql_statements.append(f"INSERT INTO {table_name} (")
        sql_statements.append("    ma_giang_vien, ho_ten, khoa_bo_mon, ngay_sinh, noi_sinh,")
        sql_statements.append("    cccd, ngay_cap_cccd, noi_cap_cccd, dan_toc, ton_giao,")
        sql_statements.append("    so_dien_thoai, dia_chi_thuong_tru, tinh_thanh_pho, phuong_xa")
        sql_statements.append(") VALUES")
        
        values = []
        for _, row in batch_df.iterrows():
            value = f"('{row['ma_giang_vien']}', '{row['ho_ten'].replace("'", "''")}', '{row['khoa_bo_mon']}', '{row['ngay_sinh']}', '{row['noi_sinh'].replace("'", "''")}', '{row['cccd']}', '{row['ngay_cap_cccd']}', '{row['noi_cap_cccd'].replace("'", "''")}', '{row['dan_toc']}', '{row['ton_giao']}', '{row['so_dien_thoai']}', '{row['dia_chi_thuong_tru'].replace("'", "''")}', '{row['tinh_thanh_pho'].replace("'", "''")}', '{row['phuong_xa'].replace("'", "''")}')"
            values.append(value)
        
        sql_statements.append(",\n".join(values) + ";")
        sql_statements.append("")
    
    return "\n".join(sql_statements)

# Generate SQL
sql_content = generate_sql_inserts(teachers_df)

# Save SQL file
sql_file = '../sql/insert_giang_vien.sql'
os.makedirs(os.path.dirname(sql_file), exist_ok=True)

with open(sql_file, 'w', encoding='utf-8') as f:
    f.write(sql_content)

print(f"SQL INSERT statements saved to: {sql_file}")
print(f"SQL file size: {os.path.getsize(sql_file) / 1024:.1f} KB")

# Show preview of SQL
print("\nSQL Preview (first 10 lines):")
print("\n".join(sql_content.split('\n')[:15]))

In [ ]:
# Final verification and summary
print("=== FINAL VERIFICATION ===")

# Verify all departments are covered
original_departments = set(khoa_bo_mon_counts.index)
generated_departments = set(teachers_df['khoa_bo_mon'].unique())

print(f"Original departments: {len(original_departments)}")
print(f"Generated departments: {len(generated_departments)}")
print(f"Missing departments: {original_departments - generated_departments}")
print(f"Extra departments: {generated_departments - original_departments}")

# Verify teacher ID sequence
teacher_ids = sorted([int(tid) for tid in teachers_df['ma_giang_vien']])
expected_ids = list(range(TEACHER_ID_START, TEACHER_ID_START + TOTAL_TEACHERS))
print(f"\nTeacher ID verification:")
print(f"Expected range: {expected_ids[0]} - {expected_ids[-1]}")
print(f"Generated range: {teacher_ids[0]} - {teacher_ids[-1]}")
print(f"Sequence is continuous: {teacher_ids == expected_ids}")

# Final file list
print(f"\n=== GENERATED FILES ===")
print(f"1. CSV data file: {output_file}")
print(f"2. SQL insert file: {sql_file}")
print(f"\nData generation completed successfully!")
print(f"Total: {len(teachers_df)} teachers across {len(generated_departments)} departments")